# OCR with llama:vision

In [9]:
import ollama
import os

In [ ]:
image_path = os.path.join(os.getcwd(), "pics", "invoice-template.webp") # Replace with your image path

# Use Ollama to analyze the image with Llama 3.2-Vision
response = ollama.chat(
    model="llama3.2-vision",
    messages=[{
      "role": "user",
      "content": "Analyze the text in the provided image. Extract all readable content and present it in a structured [Markdown/JSON] format.",
      "images": [image_path]
    }],
)

In [13]:
# Extract the model's response about the image
cleaned_text = response['message']['content'].strip()
print(f"Model Response: {cleaned_text}")

Model Response: The image shows an invoice for a brand consulting service. The invoice is for $440 and includes the following services:

*   Brand consultation
*   Logo design
*   Website design
*   Social media templates
*   Brand photography
*   Brand guide

The invoice is dated November 2, 2030, and is addressed to Richard Sanchez at Thynk Unlimited. The payment method is a bank transfer to Borcele Bank, and the account details are provided. The invoice includes a description of the services provided, the unit price, and the total amount due. The invoice also includes a tax rate of 10% and a subtotal of $400. The total amount due is $440, which includes the tax. The invoice is signed by Adeline Palmerston, who is likely the owner of the brand consulting service. Overall, the invoice provides a clear and detailed breakdown of the services provided and the amount due. It is a standard business document that is used to request payment for goods or services.
